# 📦 Como Configurar os Datasets no Kaggle

Este notebook requer alguns datasets com credenciais para funcionar corretamente. Siga os passos abaixo:

## 1️⃣ client_secret.json (Obrigatório)
Credenciais OAuth do Google Cloud para autenticação e upload no Drive.

**Como obter:**
1. Acesse [Google Cloud Console](https://console.cloud.google.com/)
2. Crie um novo projeto ou selecione um existente
3. Vá em **APIs & Services** → **Credentials**
4. Clique em **Create Credentials** → **OAuth 2.0 Client ID**
5. Escolha **Desktop App** como tipo de aplicação
6. Baixe o arquivo JSON
7. No Kaggle: **Add Data** → **Upload** → Faça upload do `client_secret.json`
8. Nomeie o dataset como `client-secret-json`

## 2️⃣ cookie (Opcional)
Cookies do YouTube/Kaggle para download de vídeos privados/restritos.

**Como obter:**
1. Instale a extensão **Get cookies.txt LOCAL** no Chrome/Edge
   - [Chrome Web Store](https://chrome.google.com/webstore/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
2. Acesse [www.kaggle.com](https://www.kaggle.com) (logado)
3. Clique na extensão e em **Export** ou **Export As**
4. Salve o arquivo como `www.youtube.com_cookies.txt`
5. No Kaggle: **Add Data** → **Upload** → Faça upload do arquivo
6. Nomeie o dataset como `cookie`

## 3️⃣ credenciais-google (Obrigatório)
API Key do Google Gemini para análise de vídeos com IA.

**Como obter:**
1. Acesse [Google AI Studio](https://makersuite.google.com/app/apikey)
2. Clique em **Create API Key**
3. Copie a chave gerada
4. Crie um arquivo de texto `gemini_api.txt` e cole a chave
5. No Kaggle: **Add Data** → **Upload** → Faça upload do arquivo
6. Nomeie o dataset como `credenciais-google`

## 4️⃣ google-drive-credentials (Gerado Automaticamente)
Este arquivo é gerado durante a autenticação OAuth na primeira execução.

**Como usar:**
1. Na primeira execução, siga as instruções de autenticação
2. **Após a autenticação bem-sucedida**, o notebook gerará o arquivo `kaggle-viral-cutte-*.json`
3. Para reutilizar nas próximas execuções:
   - Baixe o arquivo gerado em `/kaggle/working/`
   - Crie um novo dataset no Kaggle com este arquivo
   - Nomeie como `google-drive-credentials`

---

## ✅ Checklist Final
Antes de executar, verifique se você adicionou aos datasets:
- ✅ `client_secret.json` → dataset `client-secret-json`
- ⚠️ `www.youtube.com_cookies.txt` → dataset `cookie` (opcional)
- ✅ `gemini_api.txt` → dataset `credenciais-google`
- ℹ️ `kaggle-viral-cutte-*.json` → dataset `google-drive-credentials` (opcional, para reuso)

## 🎬 Pronto para Começar!
Após configurar os datasets, execute a célula abaixo para iniciar o ViralCutter.

---

In [ ]:
#@title 🚀 VIRALCUTTER: Instalação + Upload OAuth (Versão Final v3 + Faster-Whisper)
import os
import time
import threading
import json
import shutil
import subprocess
import sys
from IPython.display import clear_output

# ==============================================================================
# 1. SETUP DE BIBLIOTECAS (FIX AUTH - DOWNGRADE FORCE)
# ==============================================================================
print("1️⃣ Ajustando bibliotecas de autenticação...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "google-auth-oauthlib==0.4.6", "google-api-python-client", "--force-reinstall", "--no-deps", "-q"],
        check=True
    )
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "google-auth>=1.0.0", "requests-oauthlib>=0.7.0", "-q"], 
        check=True
    )
except Exception as e:
    print(f"⚠️ Aviso na instalação de libs: {e}")

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# ==============================================================================
# 2. INSTALAÇÃO DO SISTEMA (AUTO-REPAIR)
# ==============================================================================
VENV_PYTHON = '/kaggle/working/ViralCutter/.venv/bin/python'

if not os.path.exists(VENV_PYTHON):
    print("\n📦 Sistema não detectado. Instalando ViralCutter (3-5 min)...")
    
    if os.path.exists("/kaggle/working/ViralCutter"):
        shutil.rmtree("/kaggle/working/ViralCutter")
    
    subprocess.run("git clone https://github.com/RafaelGodoyEbert/ViralCutter.git /kaggle/working/ViralCutter", shell=True, check=True)
    os.chdir("/kaggle/working/ViralCutter")
    
    print("   - Instalando drivers...")
    subprocess.run("pip install uv -q", shell=True)
    subprocess.run("apt-get update -y -qq && apt-get install -y libcudnn8 ffmpeg xvfb -qq", shell=True)
    
    print("   - Criando ambiente virtual...")
    subprocess.run(["uv", "venv", ".venv"], check=True)
    
    print("   - Baixando pacotes de IA...")
    cmds = [
        "uv pip install --python .venv faster-whisper",  # <--- ÚNICA MUDANÇA: Adiciona motor otimizado
        "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
        "uv pip install --python .venv -r requirements.txt",
        "uv pip install --python .venv -U --pre 'yt-dlp[default]'",
        "uv pip install --python .venv pytubefix google-generativeai pandas onnxruntime-gpu",
        "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0",
        "uv pip install --python .venv torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121",
        "uv pip install --python .venv 'numpy<2.0' setuptools==69.5.1",
        "uv pip install --python .venv insightface onnxruntime-gpu",
        "uv pip uninstall --python .venv mediapipe protobuf flatbuffers",
        "uv pip install --python .venv 'mediapipe>=0.10.0' 'protobuf>=3.20,<5.0' 'flatbuffers>=2.0'"
    ]
    
    for cmd in cmds:
        subprocess.run(cmd, shell=True, check=True)
        
    print("✅ Instalação Concluída!")
    print("   💡 Faster-Whisper instalado: Agora você pode usar Large sem estourar memória!")
else:
    print("\n✅ Sistema já instalado. Pulando etapa.")

# ==============================================================================
# 3. PATCH NO DOWNLOADER
# ==============================================================================
print("\n🔧 Aplicando patch no sistema de download...")
download_script_content = r'''
import yt_dlp
import os
import glob
import shutil

def download(url, download_subs=False, quality="best", output_folder="VIRALS"):
    print(f"🚀 Iniciando download: {url}")
    title_temp = "video_temp"
    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            info = ydl.extract_info(url, download=False)
            if info:
                t = info.get('title', 'video_temp')
                title_temp = "".join([c for c in t if c.isalnum() or c in " .-_"]).strip()
    except: pass

    final_path = os.path.join(output_folder, title_temp)
    os.makedirs(final_path, exist_ok=True)
    output_template = f"{final_path}/input.%(ext)s"

    ydl_opts = {
        'format': "bestvideo+bestaudio/best",
        'outtmpl': output_template,
        'noplaylist': True,
        'writesubtitles': False,
        'writeautomaticsub': False,
        'extractor_args': {'youtube': {'player_client': ['android', 'web']}}, 
        'quiet': False,
        'no_warnings': True,
        'merge_output_format': 'mp4'
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        
        mp4_files = glob.glob(os.path.join(final_path, "*.mp4"))
        if mp4_files:
            found_file = mp4_files[0]
            print(f"✅ Arquivo baixado: {found_file}")
            expected_input = os.path.join(final_path, "input.mp4")
            if found_file != expected_input:
                shutil.move(found_file, expected_input)
                return expected_input
            return found_file
        raise FileNotFoundError("Arquivo MP4 não gerado")
    except Exception as e:
        print(f"❌ Falha crítica no Download: {e}")
        raise e
'''
with open('/kaggle/working/ViralCutter/scripts/download_video.py', 'w') as f:
    f.write(download_script_content)

# ==============================================================================
# 4. AUTENTICAÇÃO OAUTH (MANTIDO 100% ORIGINAL)
# ==============================================================================
CLIENT_SECRET_FILE = None
SCOPES = ['https://www.googleapis.com/auth/drive.file']
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'

print("\n🔑 Procurando 'client_secret.json'...")
for root, dirs, files in os.walk('/kaggle/input'):
    for file in files:
        if 'client_secret' in file and file.endswith('.json'):
            CLIENT_SECRET_FILE = os.path.join(root, file)
            print(f"✅ Credencial encontrada: {CLIENT_SECRET_FILE}")
            break
    if CLIENT_SECRET_FILE: break

service = None
folder_id = None

if CLIENT_SECRET_FILE:
    print("\n🌐 INICIANDO LOGIN GOOGLE...")
    print("   ⚠️ Siga os passos abaixo com atenção:")
    print("   1. Clique no link que aparecerá abaixo.")
    print("   2. Faça login e autorize o acesso.")
    print("   3. Copie o código gerado pelo Google.")
    print("   4. COLE O CÓDIGO na caixa de entrada aqui no Kaggle e aperte Enter.\n")
    
    try:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
        
        if hasattr(flow, 'run_console'):
            creds = flow.run_console()
        else:
            print("❌ AVISO CRÍTICO: O downgrade da biblioteca falhou.")
            print("   Tentando método alternativo via OOB...")
            flow.redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'
            auth_url, _ = flow.authorization_url(prompt='consent')
            print(f"   Acesse este URL: {auth_url}")
            code = input("   Cole o código de autorização aqui: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds)
        print("\n✅ Autenticação realizada com SUCESSO!")
        
        try:
            results = service.files().list(q="name='ViralCutter_Videos' and mimeType='application/vnd.google-apps.folder' and trashed=false", fields="files(id)").execute()
            items = results.get('files', [])
            if items:
                folder_id = items[0]['id']
                print(f"✅ Pasta 'ViralCutter_Videos' encontrada (ID: {folder_id})")
            else:
                file_metadata = {'name': 'ViralCutter_Videos', 'mimeType': 'application/vnd.google-apps.folder'}
                folder = service.files().create(body=file_metadata, fields='id').execute()
                folder_id = folder.get('id')
                print(f"✅ Pasta criada (ID: {folder_id})")
        except Exception as e:
            print(f"⚠️ Erro ao criar pasta: {e}")
    except Exception as e:
        print(f"❌ Erro no Login: {e}")
        print("   Nota: Se aparecer 'invalid_grant' ou 'redirect_uri_mismatch', suas credenciais podem não ser do tipo 'Desktop App'.")
else:
    print("❌ 'client_secret.json' não encontrado. O upload será desativado.")

# ==============================================================================
# 5. MONITOR DE UPLOAD OAUTH (MANTIDO 100% ORIGINAL)
# ==============================================================================
def monitor_oauth():
    if not service or not folder_id: 
        return
    
    print("\n👀 Monitor OAuth Ativo: Uploads usarão SEU espaço.")
    uploaded = set()
    
    while True:
        try:
            watch_path = '/kaggle/working/ViralCutter/VIRALS'
            if os.path.exists(watch_path):
                for r, d, f in os.walk(watch_path):
                    for file in f:
                        if not file.endswith('.mp4'):
                            continue
                        
                        ignore_patterns = [
                            'input.mp4', '_original_scale', '_subtitled', '_temp', 
                            'temp_', 'final_', '.part', 'processing'
                        ]
                        
                        if any(pattern in file.lower() for pattern in ignore_patterns):
                            continue
                        
                        path = os.path.join(r, file)
                        
                        if path not in uploaded and os.path.getsize(path) > 1e6:
                            s1 = os.path.getsize(path)
                            time.sleep(5)
                            if os.path.getsize(path) != s1:
                                continue
                            
                            print(f"\n💎 Novo Corte FINAL Detectado: {file}")
                            print(f"   📁 Tamanho: {os.path.getsize(path) / 1e6:.2f} MB")
                            print(f"   ☁️ Enviando para Google Drive (Conta Pessoal)...")
                            
                            try:
                                meta = {'name': file, 'parents': [folder_id]}
                                media = MediaFileUpload(path, resumable=True)
                                service.files().create(body=meta, media_body=media).execute()
                                uploaded.add(path)
                                print("   ✅ Upload Completo!")
                            except Exception as e_up:
                                print(f"   ❌ Erro Upload: {e_up}")
                                time.sleep(5)
            
            time.sleep(10)
        except Exception as e:
            print(f"⚠️ Erro no monitor: {e}")
            time.sleep(10)

threading.Thread(target=monitor_oauth, daemon=True).start()

# ==============================================================================
# 6. INICIAR (MANTIDO 100% ORIGINAL)
# ==============================================================================
print("\n🚀 INICIANDO VIRALCUTTER...")
print("⚠️ CLIQUE NO LINK PÚBLICO (gradio.live) ABAIXO:")
print("="*60)

subprocess.Popen(
    ['Xvfb', ':1', '-screen', '0', '2560x1440x8'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(2)

os.environ['DISPLAY'] = ':1.0'
os.environ['MPLBACKEND'] = 'Agg'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

!/kaggle/working/ViralCutter/.venv/bin/python webui/app.py --colab

1️⃣ Ajustando bibliotecas de autenticação...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 108.7 MB/s eta 0:00:00

📦 Sistema não detectado. Instalando ViralCutter (3-5 min)...


Cloning into '/kaggle/working/ViralCutter'...


   - Instalando drivers...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 84.6 MB/s eta 0:00:00


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Selecting previously unselected package libcudnn8.
(Reading database ... 129073 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.9.7.29-1+cuda12.2_amd64.deb ...
Unpacking libcudnn8 (8.9.7.29-1+cuda12.2) ...
Setting up libcudnn8 (8.9.7.29-1+cuda12.2) ...
   - Criando ambiente virtual...


...

   - Baixando pacotes de IA...


Resolved 30 packages in 261ms
Prepared 18 packages in 1.58s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 30 packages in 213ms
 + anyio==4.12.1
 + av==16.1.0
 + certifi==2026.1.4
 + click==8.3.1
 + coloredlogs==15.0.1
 + ctranslate2==4.7.1
 + faster-whisper==1.2.1
 + filelock==3.20.3
 + flatbuffers==25.12.19
 + fsspec==2026.1.0
 + h11==0.16.0
 + hf-xet==1.2.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + huggingface-hub==1.4.0
 + humanfriendly==10.0
 + idna==3.11
 + mpmath==1.3.0
 + numpy==2.4.2
 + onnxruntime==1.23.2
 + packaging==26.0
 + protobuf==6.33.5
 + pyyaml==6.0.3
 + setuptools==80.10.2
 + shellingham==1.5.4
 + sympy==1.14.0
 + tokenizers==0.22.2
 + tqdm==4.67.3
 + typer-slim==0.21.1
 + typing-extensions==4.15.0
Resolved 117 packages in 4.83s
Prepared 72 packages in 46.44s
Uninstalled 1 package in 

✅ Instalação Concluída!
   💡 Faster-Whisper instalado: Agora você pode usar Large sem estourar memória!

🔧 Aplicando patch no sistema de download...

🔑 Procurando 'client_secret.json'...
✅ Credencial encontrada: /kaggle/input/client-secret-json/client_secret.json

🌐 INICIANDO LOGIN GOOGLE...
   ⚠️ Siga os passos abaixo com atenção:
   1. Clique no link que aparecerá abaixo.
   2. Faça login e autorize o acesso.
   3. Copie o código gerado pelo Google.
   4. COLE O CÓDIGO na caixa de entrada aqui no Kaggle e aperte Enter.

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=6470460908-46mrv6m55s52s5cuj7nioqm8t57s1n4l.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=Oqb7A4mqoRs7NyrOH0jH12QqsNFujR&prompt=consent&access_type=offline


Enter the authorization code:  4/1ASc3gC1FY0jcZpMngnkjlzX6xvInHKlu_BgLYqgYMP0nb_nMOIdu6WVg2Ho



✅ Autenticação realizada com SUCESSO!
✅ Pasta 'ViralCutter_Videos' encontrada (ID: 1dkq555mhQkzDRe7DAN3v1mcZ6YqLt0cG)

👀 Monitor OAuth Ativo: Uploads usarão SEU espaço.

🚀 INICIANDO VIRALCUTTER...
⚠️ CLIQUE NO LINK PÚBLICO (gradio.live) ABAIXO:
Running in Colab mode. Generating public link with Static Mounts...
DEBUG: Registered static paths: ['/kaggle/working/ViralCutter/VIRALS', '/kaggle/working/ViralCutter', '/kaggle/working/ViralCutter', '.']
DEBUG: Allowed paths for Gradio: ['/kaggle/working/ViralCutter/VIRALS', '/kaggle/working/ViralCutter', '/kaggle/working/ViralCutter', '.']
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e6e28ea40b0b9652dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Mounted /virals to /kaggle/working/ViralCutter/VIRALS

💎 Novo Corte FINAL Detectado: 000_